# The Vanishing/Exploding Gradients Problems
 * The vanishing gradients problem occurs when the gradients of the loss function with respect to the parameters become extremely small during backpropagation.
 * Conversely, the exploding gradients problem arises when the gradients of the loss function grow exponentially as they propagate backward through the layers during backpropagation.


## Weight Initialization
 * Proper initialization of network weights can help alleviate both vanishing and exploding gradients problems. 
 * Techniques like __Xavier__ initialization and __He__ initialization are commonly used to ensure that the weights are initialized in a way that keeps the signal propagated through the network.

In [14]:
# Importing essential package
import numpy as np
import tensorflow as tf
from tensorflow import keras

# List comprehension to get the names of initializers in Keras without underscores
initializer_names = [name for name in dir(keras.initializers) if not name.startswith("_")]

# Printing the initializer names
print(initializer_names)

['Constant', 'GlorotNormal', 'GlorotUniform', 'HeNormal', 'HeUniform', 'Identity', 'Initializer', 'LecunNormal', 'LecunUniform', 'Ones', 'Orthogonal', 'RandomNormal', 'RandomUniform', 'TruncatedNormal', 'VarianceScaling', 'Zeros', 'constant', 'deserialize', 'get', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform', 'identity', 'lecun_normal', 'lecun_uniform', 'ones', 'orthogonal', 'random_normal', 'random_uniform', 'serialize', 'truncated_normal', 'variance_scaling', 'zeros']


* Creating a dense layer with built-in initializers

In [15]:
# Creating a dense layer with 10 units, ReLU activation, and He normal weight initialization
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

* Defining an initializer with Variance Scaling (custom kernel initializer)

In [16]:
# Defining an initializer with Variance Scaling
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')

# Creating a dense layer with 10 units, ReLU activation, and custom kernel initializer
dense_layer = keras.layers.Dense(10, activation="relu", kernel_initializer=init)

## Nonsaturating Activation Functions
 * Non-saturating activation functions refer to activation functions that do not suffer from the vanishing gradient problem to the same extent as traditional saturating activation functions like sigmoid and tanh. These functions typically have gradients that do not diminish as quickly as the __sigmoid__ or __tanh__ functions for large input values, thereby alleviating the vanishing gradient problem. 
 * One prominent example of a non-saturating activation function is the Rectified Linear Unit (__ReLU__) and its variants.

1. __Rectified Linear Unit (ReLU):__
    * ReLU is a popular activation function in deep learning, defined as __f(x)=max(0,x)__. It outputs zero for negative inputs and passes positive inputs unchanged. 
    * It's computationally efficient and doesn't suffer from vanishing gradients for positive inputs. However, it may lead to the __"dying ReLU"__ problem, causing neurons to become inactive for negative inputs. 
    * Variants like __Leaky ReLU__, __Parametric ReLU (PReLU)__, and __Exponential Linear Unit (ELU)__ address this issue by allowing a small, non-zero gradient for negative inputs.

In [17]:
# Listing ReLU variants in Keras.layers module
[m for m in dir(keras.layers) if "relu" in m.lower()]

['LeakyReLU', 'PReLU', 'ReLU', 'ThresholdedReLU']

2. __Leaky ReLU:__
    * Leaky ReLU is an activation function similar to ReLU but with a small, non-zero gradient for negative input values. 
    * It's defined as __f(x)=max(αx,x)__, where α is a small constant (e.g., 0.01). 
    * Leaky ReLU addresses the __"dying ReLU"__ problem by preventing neurons from becoming inactive for negative inputs during training. It retains the efficiency of ReLU while mitigating its limitations.

In [18]:
# Training a neural network on Fashion MNIST using the Leaky ReLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Defining the neural network model with Leaky ReLU activation
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),  # Leaky ReLU activation
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),  # Leaky ReLU activation
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 2s 1ms/step - loss: 1.2620 - accuracy: 0.6082 - val_loss: 0.8597 - val_accuracy: 0.7190
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.7782 - accuracy: 0.7447 - val_loss: 0.6975 - val_accuracy: 0.7692
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6708 - accuracy: 0.7790 - val_loss: 0.6341 - val_accuracy: 0.7958
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6145 - accuracy: 0.7984 - val_loss: 0.5828 - val_accuracy: 0.8104
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5778 - accuracy: 0.8094 - val_loss: 0.5528 - val_accuracy: 0.8186
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5513 - accuracy: 0.8156 - val_loss: 0.5306 - val_accuracy: 0.8242
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5312 - accuracy: 0.8225 - val_loss: 0.5123 - val_accuracy:

3. __Parametric ReLU:__
    * Parametric ReLU (PReLU) extends Leaky ReLU by allowing the coefficient α to be learned during training rather than fixed. PReLU can adaptively adjust the leakage for negative inputs based on data, potentially improving performance over fixed coefficients.

In [19]:
# Training a neural network on Fashion MNIST using the Leaky PReLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Defining the neural network model with Leaky ReLU activation
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.3554 - accuracy: 0.5967 - val_loss: 0.9191 - val_accuracy: 0.7148
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.8232 - accuracy: 0.7405 - val_loss: 0.7281 - val_accuracy: 0.7724
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6964 - accuracy: 0.7779 - val_loss: 0.6504 - val_accuracy: 0.7978
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6292 - accuracy: 0.7960 - val_loss: 0.5905 - val_accuracy: 0.8104
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5862 - accuracy: 0.8085 - val_loss: 0.5552 - val_accuracy: 0.8222
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5558 - accuracy: 0.8150 - val_loss: 0.5302 - val_accuracy: 0.8316
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5332 - accuracy: 0.8220 - val_loss: 0.5093 - val_accuracy:

4. __Exponential Linear Unit (ELU):__
    * ELU is defined as f(x)=x if x≥0 and f(x)=α(e^x−1) if x<0, where α is a small positive constant.
    * ELU behaves similarly to ReLU for positive input values but has a smooth curve for negative input values, allowing it to handle negative inputs more gracefully than ReLU. 
    * ELU also prevents the "dying ReLU" problem by ensuring a non-zero gradient for all inputs.

In [20]:
# Training a neural network on Fashion MNIST using the Leaky PReLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Defining the neural network model with Leaky ReLU activation
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal", activation="elu"),
    keras.layers.Dense(100, kernel_initializer="he_normal", activation="elu"),
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.1755 - accuracy: 0.6214 - val_loss: 0.7931 - val_accuracy: 0.7412
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.7299 - accuracy: 0.7599 - val_loss: 0.6552 - val_accuracy: 0.7842
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6371 - accuracy: 0.7885 - val_loss: 0.6008 - val_accuracy: 0.7980
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5883 - accuracy: 0.8020 - val_loss: 0.5559 - val_accuracy: 0.8164
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5569 - accuracy: 0.8110 - val_loss: 0.5310 - val_accuracy: 0.8236
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5345 - accuracy: 0.8173 - val_loss: 0.5118 - val_accuracy: 0.8282
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5175 - accuracy: 0.8226 - val_loss: 0.4964 - val_accuracy:

5. __Scaled Exponential Linear Unit (SELU):__
    * It's an activation function introduced to address the vanishing/exploding gradient problem while promoting self-normalizing properties in neural networks.
    * SELU has a self-normalizing property, meaning the output distribution remains close to mean 0 and standard deviation 1, which helps stabilize training.
    * It overcomes the vanishing/exploding gradient problem by promoting stable mean and variance propagation through the network.
    * It has an exponential component for negative inputs, ensuring non-zero gradients, thus avoiding the "dying ReLU" problem.
    * SELU is specifically designed for feedforward neural networks, and its performance may vary depending on the architecture and hyperparameters.

In [21]:
# Training a neural network on Fashion MNIST using the Leaky SeLU:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
# Scaling pixel values to the range [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
# Splitting training set into validation and training subsets
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Calculating mean and standard deviation of training pixel values
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
# Standardizing the input features using mean and standard deviation
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Constructing the neural network model with SELU activation
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"))
# Adding 99 hidden layers with SELU activation
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))
# Adding output layer with softmax activation for 10 classes
model.add(keras.layers.Dense(10, activation="softmax"))

# Compiling the model with sparse categorical crossentropy loss and SGD optimizer
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model on scaled training data for 5 epochs with validation data for monitoring
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 40s 22ms/step - loss: 1.3773 - accuracy: 0.4634 - val_loss: 1.0353 - val_accuracy: 0.6122
Epoch 2/5
1719/1719 [==============================] - 37s 21ms/step - loss: 0.8848 - accuracy: 0.6701 - val_loss: 0.7549 - val_accuracy: 0.7258
Epoch 3/5
1719/1719 [==============================] - 37s 21ms/step - loss: 0.7550 - accuracy: 0.7158 - val_loss: 0.6541 - val_accuracy: 0.7418
Epoch 4/5
1719/1719 [==============================] - 37s 21ms/step - loss: 0.6429 - accuracy: 0.7578 - val_loss: 0.5857 - val_accuracy: 0.7784
Epoch 5/5
1719/1719 [==============================] - 37s 21ms/step - loss: 0.5964 - accuracy: 0.7803 - val_loss: 0.5613 - val_accuracy: 0.7952


## Batch Normalization
 * Batch Normalization is a technique used in deep learning to enhance the training process of neural networks. 
 * This means that it adjusts the activations of each layer so that they have a mean of zero and a standard deviation of one. By doing this normalization, Batch Normalization helps to stabilize the training process and improve the speed at which neural networks converge to an optimal solution.

 * Batch Normalization has become a standard technique in deep learning because it contributes to __faster training__, __improved model performance__, and __greater stability__ during the training process.

In [22]:
# Training a neural network on Fashion MNIST using the Batch Normalization:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
# Scaling pixel values to the range [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
# Splitting training set into validation and training subsets
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Define a Sequential model
model = keras.models.Sequential([
    # Flatten layer to convert 2D input into 1D
    keras.layers.Flatten(input_shape=[28, 28]),
    # BatchNormalization layer applied after the Flatten layer
    keras.layers.BatchNormalization(),
    # Dense layer with 300 neurons and ReLU activation function
    keras.layers.Dense(300, activation="relu"),
    # BatchNormalization layer applied after the first Dense layer
    keras.layers.BatchNormalization(),
    # Dense layer with 100 neurons and ReLU activation function
    keras.layers.Dense(100, activation="relu"),
    # BatchNormalization layer applied after the second Dense layer
    keras.layers.BatchNormalization(),
    # Dense layer with 10 neurons and softmax activation function for classification
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model with sparse categorical crossentropy loss and SGD optimizer
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model on training data for 10 epochs with validation data for monitoring
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8608 - accuracy: 0.7107 - val_loss: 0.5570 - val_accuracy: 0.8092
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5727 - accuracy: 0.7987 - val_loss: 0.4779 - val_accuracy: 0.8330
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5201 - accuracy: 0.8180 - val_loss: 0.4436 - val_accuracy: 0.8444
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4815 - accuracy: 0.8307 - val_loss: 0.4197 - val_accuracy: 0.8510
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4577 - accuracy: 0.8387 - val_loss: 0.4058 - val_accuracy: 0.8598
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4422 - accuracy: 0.8442 - val_loss: 0.3926 - val_accuracy: 0.8622
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4264 - accuracy: 0.8496 - val_loss: 0.3826 - val_accuracy:

In [23]:
# Display a summary of the model architecture
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_11 (Flatten)        (None, 784)               0         
                                                                 
 batch_normalization_9 (Batc  (None, 784)              3136      
 hNormalization)                                                 
                                                                 
 dense_233 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_10 (Bat  (None, 300)              1200      
 chNormalization)                                                
                                                                 
 dense_234 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_11 (Bat  (None, 100)            

In [24]:
# Retrieve the BatchNormalization layer after the first layer in the model
bn1 = model.layers[1]

# List the names and trainability of the variables in the BatchNormalization layer
# This can be useful for inspecting the variables of the BatchNormalization layer
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization_9/gamma:0', True),
 ('batch_normalization_9/beta:0', True),
 ('batch_normalization_9/moving_mean:0', False),
 ('batch_normalization_9/moving_variance:0', False)]

### Batch Normalization: Before or After Activation?
* Applying Batch Normalization (BN) __before__ the __activation function__ is a debated topic in deep learning. While sometimes it yields __better results__, it's subject to discussion. Additionally, it's common practice to omit bias terms in layers preceding BatchNormalization. Since BatchNormalization introduces its own bias parameters, setting __use_bias=False__ for these preceding layers avoids redundant parameters and optimizes model efficiency.

In [25]:
# Training a neural network on Fashion MNIST using the Batch Normalization before the activation function:

# Loading Fashion MNIST dataset and preprocessing
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
# Scaling pixel values to the range [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
# Splitting training set into validation and training subsets
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


# Setting random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Define a Sequential model
model = keras.models.Sequential([
    # Flatten layer to convert 2D input into 1D
    keras.layers.Flatten(input_shape=[28, 28]),
    # BatchNormalization layer applied before the activation function
    keras.layers.BatchNormalization(),
    # Dense layer with 300 neurons and no bias term
    keras.layers.Dense(300, use_bias=False),
    # BatchNormalization layer applied before the activation function
    keras.layers.BatchNormalization(),
    # Activation function ReLU applied
    keras.layers.Activation("relu"), 
    # Dense layer with 100 neurons and no bias term
    keras.layers.Dense(100, use_bias=False),
    # BatchNormalization layer applied before the activation function
    keras.layers.BatchNormalization(),
    # Activation function ReLU applied
    keras.layers.Activation("relu"),
    # Dense layer with 10 neurons and softmax activation function for classification
    keras.layers.Dense(10, activation="softmax")
])

# Compiling the model with sparse categorical crossentropy loss and SGD optimizer
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

# Training the model on training data for 10 epochs with validation data for monitoring
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 1.0497 - accuracy: 0.6727 - val_loss: 0.6831 - val_accuracy: 0.7822
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6791 - accuracy: 0.7816 - val_loss: 0.5618 - val_accuracy: 0.8138
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5965 - accuracy: 0.8032 - val_loss: 0.5077 - val_accuracy: 0.8296
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5443 - accuracy: 0.8193 - val_loss: 0.4749 - val_accuracy: 0.8384
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5111 - accuracy: 0.8275 - val_loss: 0.4513 - val_accuracy: 0.8462
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4892 - accuracy: 0.8334 - val_loss: 0.4330 - val_accuracy: 0.8508
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4715 - accuracy: 0.8391 - val_loss: 0.4190 - val_accuracy:

## Gradient Clipping
 * Gradient clipping addresses exploding gradients problem by imposing a constraint on the gradients during training. Specifically, it involves scaling the gradients if their norm exceeds a certain threshold.
 
 
 * __Gradient Clipping With clipvalue__: Each component of the gradient vector is individually scaled if it falls outside a specified range. This technique restricts the magnitude of each gradient component without necessarily preserving the overall direction of the gradient vector.
 
 * __Gradient Clipping With clipnorm__: The entire gradient vector is scaled down if its norm exceeds a specified threshold. This ensures that the direction of the gradient vector is preserved while preventing it from becoming too large.

  * If you want to ensure that Gradient Clipping does not change the direction of the gradient vector, you should clip by norm by setting clipnorm instead of clipvalue.

 * Gradient clipping is commonly used in recurrent neural networks (RNNs) because Batch Normalization (BN) is difficult to implement in RNNs. However, for other network types, like feedforward or convolutional neural networks (CNNs), Batch Normalization is usually sufficient.


In [ ]:
# Set optimizer with gradient clipping by clipvalue
optimizer = keras.optimizers.SGD(clipvalue=1.0)

# Set optimizer with gradient clipping by clipnorm
optimizer = keras.optimizers.SGD(clipnorm=1.0)

## Reusing Pretrained Layers